In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
import pandas as pd
from numba import njit
import logging
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_custom_logger import nb_CustomLogger, nb_WontLog
from nb_quantfreedom.nb_base import create_classes

from nb_quantfreedom.nb_enums import LeverageModeType, PositionModeType
from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product
from nb_quantfreedom.nb_helper_funcs import get_to_the_upside_nb, nb_get_dos
from nb_quantfreedom.nb_enums import (
    BacktestSettings,
    CandleBodyType,
    CandleProcessingType,
    DecreasePosition,
    DynamicOrderSettingsArrays,
    ExchangeSettings,
    IncreasePositionType,
    LeverageStrategyType,
    LoggerSettings,
    LongOrShortType,
    MoveStopLoss,
    OrderResult,
    OrderStatus,
    PriceGetterType,
    RejectedOrder,
    ZeroOrEntryType,
    StopLossStrategyType,
    TakeProfitFeeType,
    TakeProfitStrategyType,
    StaticOrderSettings,
    ZeroOrEntryType,
    strat_df_array_dt,
)
from nb_quantfreedom.nb_order_handler.nb_decrease_position import nb_DecreasePosition, nb_Long_DP
from nb_quantfreedom.nb_order_handler.nb_increase_position import (
    nb_IncreasePosition,
    nb_Long_RPAandSLB,
    nb_Long_SEP,
)
from nb_quantfreedom.nb_order_handler.nb_leverage import (
    nb_Leverage,
    nb_Long_DLev,
    nb_Long_Leverage,
    nb_Long_SLev,
)
from nb_quantfreedom.nb_order_handler.nb_class_helpers import (
    nb_GetMaxPrice,
    nb_GetMinPrice,
    nb_Long_SLToEntry,
    nb_Long_SLToZero,
    nb_PriceGetter,
    nb_ZeroOrEntry,
)

from nb_quantfreedom.nb_order_handler.nb_stop_loss import (
    nb_Long_SLBCB,
    nb_Long_StopLoss,
    nb_MoveSL,
    nb_StopLoss,
)
from nb_quantfreedom.nb_order_handler.nb_take_profit import (
    nb_Long_RR,
    nb_Long_TPHitReg,
    nb_TakeProfit,
)
from nb_quantfreedom.strategies.nb_strategy import (
    nb_BacktestInd,
    nb_CreateInd,
    nb_Strategy,
    nb_TradingInd,
)

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="5m", limit=1000)

It took 00 mins and 00 seconds to download 1000 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([3]),
    max_trades=np.array([5]),
    num_candles=np.array([60]),
    risk_account_pct_size=np.array([3, 5]),
    risk_reward=np.array([3]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([10]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([0]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1.20]),
    trail_sl_when_pct=np.array([0.001]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = nb_dos_cart_product(
    dos_arrays=dos_arrays,
)

In [5]:
(
    checker_liq_hit,
    checker_sl_hit,
    checker_sl_to_be,
    checker_tp_hit,
    checker_tsl,
    dec_pos_calculator,
    dos_cart_arrays,
    exit_fee_pct,
    inc_pos_calculator,
    ind_creator,
    lev_calculator,
    log_logging,
    set_z_e,
    sl_bcb_price_getter,
    sl_calculator,
    sl_mover,
    strategy,
    tp_calculator,
) = create_classes(
    backtest_settings=backtest_settings,
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    dos_cart_arrays=dos_cart_arrays,
    exchange_settings=mufex_main.exchange_settings,
    logger_settings=None,
    # logger_settings=LoggerSettings(log_debug=True, create_trades_logger=False),
    starting_equity=1000.0,
    static_os=static_os,
)

In [6]:
@njit()
def testingme(log_logging: nb_CustomLogger = log_logging):
    return log_logging.log_debug(6)

In [7]:
testingme()

NumbaNotImplementedError: Failed in nopython mode pipeline (step: native lowering)
[1m[1m<numba.core.base.OverloadSelector object at 0x00000210FBA94C70>, (instance.jitclass.nb_WontLog#210fba97b80<>,)[0m
[0m[1mDuring: lowering "log_logging = arg(0, name=log_logging)" at C:\Users\User\AppData\Local\Temp\ipykernel_13184\3834717069.py (1)[0m

In [ ]:
logger.debug("hey")

In [ ]:
cbt = 4
price = 10
hello = "nb_class_helpers.py - nb_GetPrice - nb_price_getter() - candle_body_type=" + str(cbt) + " price min=" + str(price)
logger.debug(hello)

In [ ]:
@njit(cache=True)
def printtest():
    print("nb_class_helpers.py - nb_GetPrice - nb_price_getter() - candle_body_type={cbt} price min={price}".format(
    cbt=6, price=5
))

In [ ]:
logger.debug('hey')